### Setup

In [2]:
os.system("pip install twscrape")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


0

In [3]:
import os
import random
from typing import List
import twscrape

### Case 1: Scraping general tweet set for given year

In [4]:
def set_up_directories(keyword_eng: str, year: str):
    dir_path = year if keyword_eng == "" else keyword_eng + "_" + year
    os.mkdir(dir_path)

    # single digit month
    for i in range(1,10):
        path = dir_path + "/" + "0" + str(i)
        os.mkdir(path) 

    # double digit month
    for j in range(10,13):
        path = dir_path + "/" + str(j)
        os.mkdir(path) 

In [5]:
# scrape randomly sampled tweets for 10 days in given month
def scrape_month_sampled_tweets(year: str, months: List[str], days_in_month: int):
    range_days = list(range(1,days_in_month+1))
    range_times = list(range(0,24))

    # sample 10 random days and times of day for each month
    for m in months: 
        month_path = year + "/" + m
        days = sorted(random.sample(range_days, k=10))   # random days of month (no replacement)
        times = random.choices(range_times, k=10)   # random hours of day (replacement)
        
        # scrape tweets for the 10 picked days and times
        for t in range(10):     
            day = "0" + str(days[t]) if days[t] < 10 else str(days[t])
            time = "0" + str(times[t]) if times[t] < 10 else str(times[t])
            date_string = year + '-' + m + '-' + day
            day_path = month_path + "/" + year + "-" + m + "-" + day + ".txt"
            command = 'twscrape search "since:' + date_string + '_' + time + ':00:00_UTC until:' + date_string + '_' + time + ':59:59_UTC lang:ja"　> ' + day_path + ' --limit=4500'
            os.system(command)

In [6]:
# scrape random sample of tweets (general content or keyword-search) from given year
def scrape_year_sampled_tweets(year: str):
    # 28 day months
    scrape_month_sampled_tweets(year,["02"], 28)    # omit leap year 29th days for simplicity...?

    # 30 day months
    months_30 = ["04","06","09","11"]
    scrape_month_sampled_tweets(year,months_30, 30)

    # 31 day months
    months_31 = ["01","03","05","07","08","10","12"]
    scrape_month_sampled_tweets(year,months_31, 31)

In [7]:
def concatenate_general_txt_files(year: str, keyword_eng=""):
  # concatenate .txt files into one file per month
  for root, dirs, files in os.walk("./" + year):
      for name in dirs:
        month_path = os.path.join(root, name)
        os.system("cat " + month_path + "/*.txt > " + year + "/" + name + ".txt")
  
  # concatenate month .txt files into one file for the year
  os.system("cat " + year + "/*.txt > " + year + "-all.txt")

#### Scraping general year sample datasets

In [ ]:
# last took 1790m
scrape_year_sampled_tweets("2015")
concatenate_general_txt_files("2015")

In [8]:
scrape_year_sampled_tweets("2022")
concatenate_general_txt_files("2022")

2024-04-09 21:13:48.239 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:275 - No account available for queue "SearchTimeline". Next available at 21:25:28


### Case 2: Scraping minority keyword-related tweet set

In [6]:
# scrape tweets containing keyword from every month of given year
def scrape_keyword_month_tweets(year: str, months: List[str], days_in_month: int, keyword_jp: str, keyword_eng: str):
    for m in months: 
        command = 'twscrape search "' + keyword_jp + ' since:' + year + '-' + m + '-01_00:00:00_UTC until:' + year + '-' + m + '-' + str(days_in_month) + '_23:59:59_UTC lang:ja"　> ' + keyword_eng + '_' + year + '/' + m + '.txt --limit=30000'
        os.system(command)

In [7]:
# scrape all tweets containing keyword in given year
def scrape_keyword_sampled_tweets(year: str, keyword_jp: str, keyword_eng: str):
    os.mkdir(keyword_eng + "_" + year) 

    # 28 day months
    scrape_keyword_month_tweets(year,["02"], 28, keyword_jp, keyword_eng)    # omit leap year 29th days for simplicity...?

    # 30 day months
    months_30 = ["04","06","09","11"]
    scrape_keyword_month_tweets(year,months_30, 30, keyword_jp, keyword_eng)

    # 31 day months
    months_31 = ["01","03","05","07","08","10","12"]
    scrape_keyword_month_tweets(year,months_31, 31, keyword_jp, keyword_eng)

In [8]:
def concatenate_keyword_txt_files(year: str, keyword_eng=""):
  # concatenate month .txt files into one file for the year
  os.system("cat " + keyword_eng + "_" + year + "/" + "*.txt > " + keyword_eng + "_" + year + ".txt")

#### Minority keyword-related datasets

In [ ]:
# zainichi korean
scrape_keyword_sampled_tweets("2022", "在日コリアン", "zainichi")
concatenate_keyword_txt_files("2022", "zainichi")

scrape_keyword_sampled_tweets("2015", "在日コリアン", "zainichi")
concatenate_keyword_txt_files("2015", "zainichi")

In [ ]:
# ainu
scrape_keyword_sampled_tweets("2022", "アイヌ", "ainu")
concatenate_keyword_txt_files("2022", "ainu")

scrape_keyword_sampled_tweets("2015", "アイヌ", "ainu")
concatenate_keyword_txt_files("2015", "ainu")

In [ ]:
# ryukyujin 
scrape_keyword_sampled_tweets("2022", "琉球人", "ryukyujin")
concatenate_keyword_txt_files("2022", "ryukyujin")

scrape_keyword_sampled_tweets("2015", "琉球人", "ryukyujin")
concatenate_keyword_txt_files("2015", "ryukyujin")

In [ ]:
# not finished but didn't use
scrape_keyword_sampled_tweets("2015", "琉球", "ryukyu")
concatenate_keyword_txt_files("2015", "ryukyu")

scrape_keyword_sampled_tweets("2022", "琉球", "ryukyu")
concatenate_keyword_txt_files("2022", "ryukyu")

In [ ]:
# okinawajin
scrape_keyword_sampled_tweets("2022", "沖縄人", "okinawajin")
concatenate_keyword_txt_files("2022", "okinawajin")

scrape_keyword_sampled_tweets("2015", "沖縄人", "okinawajin")
concatenate_keyword_txt_files("2015", "okinawajin")

In [ ]:
# haafu
# this one took around 100 min per month...
scrape_keyword_sampled_tweets("2022", "ハーフ", "haafu")
concatenate_keyword_txt_files("2022", "haafu")

scrape_keyword_sampled_tweets("2015", "ハーフ", "haafu")
concatenate_keyword_txt_files("2015", "haafu")

In [ ]:
# vietnam
scrape_keyword_sampled_tweets("2022", "ベトナム人", "vietnamjin")
concatenate_keyword_txt_files("2022", "vietnamjin")

scrape_keyword_sampled_tweets("2015", "ベトナム人", "vietnamjin")
concatenate_keyword_txt_files("2015", "vietnamjin")

In [ ]:
# philippines
scrape_keyword_sampled_tweets("2022", "フィリピン人", "philippinejin")
concatenate_keyword_txt_files("2022", "philippinejin")

scrape_keyword_sampled_tweets("2015", "フィリピン人", "philippinejin")
concatenate_keyword_txt_files("2015", "philippinejin")

In [ ]:
# nepal
scrape_keyword_sampled_tweets("2022", "ネパール人", "nepaljin")
concatenate_keyword_txt_files("2022", "nepaljin")

scrape_keyword_sampled_tweets("2015", "ネパール人", "nepaljin")
concatenate_keyword_txt_files("2015", "nepaljin")

In [ ]:
# indonesia
scrape_keyword_sampled_tweets("2022", "インドネシア人", "indonesiajin")
concatenate_keyword_txt_files("2022", "indonesiajin")

scrape_keyword_sampled_tweets("2015", "インドネシア人", "indonesiajin")
concatenate_keyword_txt_files("2015", "indonesiajin")

In [ ]:
# zainichi kankokujin
scrape_keyword_sampled_tweets("2022", "在日韓国人", "zainichi_kankokujin")
concatenate_keyword_txt_files("2022", "zainichi_kankokujin")

scrape_keyword_sampled_tweets("2015", "在日韓国人", "zainichi_kankokujin")
concatenate_keyword_txt_files("2015", "zainichi_kankokujin")

In [ ]:
# gaijin [didn't do 2022; hold]
scrape_keyword_sampled_tweets("2022", "外人", "gaijin")
concatenate_keyword_txt_files("2022", "gaijin")

scrape_keyword_sampled_tweets("2015", "外人", "gaijin")
concatenate_keyword_txt_files("2015", "gaijin")